<a href="https://www.kaggle.com/code/avtnshm/clinical-modernbert-v-biomedicalmb-on-ddxplus-data?scriptVersionId=255007048" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

# Load the train, test, and validate datasets
train_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/train.csv')
test_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/test.csv')
validate_df = pd.read_csv('/kaggle/input/mldataset/ddxplus/validate.csv')

# Display the first few rows of the train dataset
train_df.head()

In [ ]:

import json

# Load JSON files
with open('/kaggle/input/mldataset/ddxplus/release_conditions.json', 'r') as f:
    conditions = json.load(f)
with open('/kaggle/input/mldataset/ddxplus/release_evidences.json', 'r') as f:
    evidences = json.load(f)

# Convert JSON to DataFrames for easier inspection (optional)
conditions_df = pd.DataFrame.from_dict(conditions, orient='index')
evidences_df = pd.DataFrame.from_dict(evidences, orient='index')

# Display the first few rows
print("Conditions (Diseases):")
print(conditions_df.head())
print("\nEvidences (Symptoms):")
print(evidences_df.head())

In [ ]:
def map_symptom_codes(symptom_dict, evidences_df):
    return [evidences_df.loc[code]["question_en"] for code in symptom_dict.keys() if code in evidences_df.index]

conditions_df["symptom_questions"] = conditions_df["symptoms"].apply(lambda x: map_symptom_codes(x, evidences_df))
conditions_df["antecedent_questions"] = conditions_df["antecedents"].apply(lambda x: map_symptom_codes(x, evidences_df))

In [ ]:
print(conditions_df[["condition_name", "symptom_questions", "antecedent_questions"]].head())

In [ ]:
# Map PATHOLOGY to condition_name
train_df['disease_name'] = train_df['PATHOLOGY'].map(lambda x: conditions.get(x, {}).get('condition_name', x))

# Display the result
print(train_df[['PATHOLOGY', 'disease_name']].head())

In [ ]:
# Delete the symptom_questions and antecedent_questions columns
conditions_df = conditions_df.drop(columns=["symptom_questions", "antecedent_questions"], errors="ignore")

# Verify the columns are removed
print(conditions_df.columns)

In [ ]:
def map_symptom_codes(symptom_dict, evidences_df):
    # Return list of English questions for valid symptom codes
    return [evidences_df.loc[code, "question_en"] for code in symptom_dict.keys() if code in evidences_df.index]

# Apply to symptoms and antecedents in conditions_df
conditions_df["symptom_questions"] = conditions_df["symptoms"].apply(lambda x: map_symptom_codes(x, evidences_df))
conditions_df["antecedent_questions"] = conditions_df["antecedents"].apply(lambda x: map_symptom_codes(x, evidences_df))

# Display the result
print(conditions_df[["condition_name", "symptom_questions", "antecedent_questions"]].head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot top 10 diseases
plt.figure(figsize=(10, 6))
train_df['disease_name'].value_counts().head(10).plot(kind='bar', color='#1f77b4')
plt.title('Top 10 Most Frequent Diseases')
plt.xlabel('Disease')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

In [38]:
# Column names from conditions_df
print("Conditions DataFrame columns:")
print(conditions_df.columns.tolist())

# Column names from evidences_df
print("\nEvidences DataFrame columns:")
print(evidences_df.columns.tolist())


print("Train columns:", train_df.columns.tolist())
print("Validate columns:", validate_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())

Conditions DataFrame columns:
['condition_name', 'cond-name-fr', 'cond-name-eng', 'icd10-id', 'symptoms', 'antecedents', 'severity', 'symptom_questions', 'antecedent_questions']

Evidences DataFrame columns:
['name', 'code_question', 'question_fr', 'question_en', 'is_antecedent', 'default_value', 'value_meaning', 'possible-values', 'data_type']
Train columns: ['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'EVIDENCES', 'INITIAL_EVIDENCE', 'disease_name']
Validate columns: ['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'EVIDENCES', 'INITIAL_EVIDENCE']
Test columns: ['AGE', 'DIFFERENTIAL_DIAGNOSIS', 'SEX', 'PATHOLOGY', 'EVIDENCES', 'INITIAL_EVIDENCE']


In [ ]:
!pip install transformers -q
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
CMB = AutoModel.from_pretrained('Simonlee711/Clinical_ModernBERT')
tokenizer1 = AutoTokenizer.from_pretrained('Simonlee711/Clinical_ModernBERT')
CMB.__class__

In [ ]:
from transformers import AutoModel

BioClinical_ModernBERT = "thomas-sounack/BioClinical-ModernBERT-base"

tokenizer2 = AutoTokenizer.from_pretrained(BioClinical_ModernBERT)
BMB = AutoModel.from_pretrained(BioClinical_ModernBERT)

BMB.__class__

In [ ]:
print(train_df['DIFFERENTIAL_DIAGNOSIS'].nunique())
print(test_df['DIFFERENTIAL_DIAGNOSIS'].nunique())
validate_df['DIFFERENTIAL_DIAGNOSIS'].nunique()

In [35]:
!nvidia-smi

Fri Aug  8 15:52:11 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
import torch
import numpy as np
import time

# Helper: get English symptom questions from evidence codes
def get_symptom_text(evidence_codes, evidences_df):
    return " ".join([
        evidences_df.loc[code]["question_en"]
        for code in evidence_codes.split(';') if code in evidences_df.index
    ])

# Take a small sample of 100 rows
sample_df = train_df.sample(n=100, random_state=42).copy()

# Create text column for embedding input
sample_df['symptom_text'] = sample_df['EVIDENCES'].apply(lambda x: get_symptom_text(x, evidences_df))

# Function to get CLS embedding from a model
def get_cls_embedding(text_list, tokenizer, model):
    model.eval()
    embeddings = []
    for text in text_list:
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
        embeddings.append(cls_embedding.squeeze().numpy())
    return np.array(embeddings)

# Generate embeddings for both models with timing
print("Generating embeddings from Clinical_ModernBERT (CMB)...")
start_cmb = time.time()
cmb_embeddings = get_cls_embedding(sample_df['symptom_text'], tokenizer1, CMB)
time_cmb = time.time() - start_cmb
print(f"CMB embedding time: {time_cmb:.2f} seconds")

print("Generating embeddings from BioClinical_ModernBERT (BMB)...")
start_bmb = time.time()
bmb_embeddings = get_cls_embedding(sample_df['symptom_text'], tokenizer2, BMB)
time_bmb = time.time() - start_bmb
print(f"BMB embedding time: {time_bmb:.2f} seconds")


Generating embeddings from Clinical_ModernBERT (CMB)...
CMB embedding time: 3.30 seconds
Generating embeddings from BioClinical_ModernBERT (BMB)...
BMB embedding time: 3.97 seconds


In [36]:
import torch
import numpy as np
import time

# Helper: get English symptom questions from evidence codes
def get_symptom_text(evidence_codes, evidences_df):
    return " ".join([
        evidences_df.loc[code]["question_en"]
        for code in evidence_codes.split(';') if code in evidences_df.index
    ])

# Take exactly 100k samples
sample_df = train_df.sample(n=100_000, random_state=42).copy()

# Create text column for embedding input
sample_df['symptom_text'] = sample_df['EVIDENCES'].apply(lambda x: get_symptom_text(x, evidences_df))

# Batched CLS embedding function
def get_cls_embedding_batched(text_list, tokenizer, model, batch_size=64):
    model.eval()
    model.to('cuda')
    all_embeddings = []

    for i in range(0, len(text_list), batch_size):
        batch_texts = text_list[i:i+batch_size]
        inputs = tokenizer(
            list(batch_texts),
            return_tensors='pt',
            truncation=True,
            padding=True,
            max_length=512
        ).to('cuda')

        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
        all_embeddings.append(cls_embeddings.cpu().numpy())

    return np.vstack(all_embeddings)

# Generate embeddings for both models with timing
print("Generating embeddings from Clinical_ModernBERT (CMB) with batching...")
start_cmb = time.time()
cmb_embeddings = get_cls_embedding_batched(sample_df['symptom_text'], tokenizer1, CMB, batch_size=64)
time_cmb = time.time() - start_cmb
print(f"CMB batch embedding time for 100k samples: {time_cmb/60:.2f} minutes")

print("\nGenerating embeddings from BioClinical_ModernBERT (BMB) with batching...")
start_bmb = time.time()
bmb_embeddings = get_cls_embedding_batched(sample_df['symptom_text'], tokenizer2, BMB, batch_size=64)
time_bmb = time.time() - start_bmb
print(f"BMB batch embedding time for 100k samples: {time_bmb/60:.2f} minutes")

# Total time
total_time = time_cmb + time_bmb
print(f"\nTotal embedding time for both models: {total_time/60:.2f} minutes")


Generating embeddings from Clinical_ModernBERT (CMB) with batching...
CMB batch embedding time for 100k samples: 0.39 minutes

Generating embeddings from BioClinical_ModernBERT (BMB) with batching...
BMB batch embedding time for 100k samples: 0.59 minutes

Total embedding time for both models: 0.98 minutes


In [40]:
# Full final cell: batched embedding for full dataset, per-batch save, and timing
import os
import time
import numpy as np
from tqdm import tqdm
import torch

# ---------------- Configuration ----------------
BATCH_SIZE = 64
OUT_DIR = "/kaggle/working/embeddings"              # final files and chunks will be saved here
os.makedirs(OUT_DIR, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ---------------- Helper: build text for each row ----------------
def get_symptom_text(evidence_codes, evidences_df):
    # safe guard if NaN or empty
    if not isinstance(evidence_codes, str) or evidence_codes.strip() == "":
        return ""
    parts = []
    for code in evidence_codes.split(';'):
        if code in evidences_df.index:
            parts.append(evidences_df.loc[code, "question_en"])
    return " ".join(parts)

def build_text_list(train_df, validate_df, test_df, evidences_df):
    # concatenate dataframes in the same order you want embeddings saved
    dfs = [train_df, validate_df, test_df]
    texts = []
    for df in dfs:
        for _, row in df.iterrows():
            # prefer EVIDENCES -> question_en mapping if present/available
            if 'EVIDENCES' in df.columns and isinstance(row.get('EVIDENCES', ""), str) and row['EVIDENCES'].strip() != "":
                txt = get_symptom_text(row['EVIDENCES'], evidences_df)
                if txt:
                    texts.append(txt)
                    continue
            # fallback to PATHOLOGY column (string)
            if 'PATHOLOGY' in df.columns and not pd.isna(row.get('PATHOLOGY', None)):
                texts.append(str(row['PATHOLOGY']))
            else:
                texts.append("")  # keep alignment for counts
    return texts

# build full_texts (ordered: train, validate, test)
import pandas as pd
full_texts = build_text_list(train_df, validate_df, test_df, evidences_df)
n_samples = len(full_texts)
print(f"Total texts to embed (train+validate+test): {n_samples:,}")

# ---------------- Batched embedding (per-batch save) ----------------
def embed_and_save_in_batches(text_list, tokenizer, model, model_tag, batch_size=64, out_dir=OUT_DIR):
    model = model.to(device)
    model.eval()
    chunk_files = []
    total_batches = (len(text_list) + batch_size - 1) // batch_size

    start_time = time.time()
    processed = 0

    for batch_idx in tqdm(range(total_batches), desc=f"{model_tag} batches"):
        i = batch_idx * batch_size
        batch_texts = text_list[i : i + batch_size]
        # tokenizer -> tensors, move to device
        inputs = tokenizer(
            list(batch_texts),
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # (batch_size, hidden_dim)

        # Save this chunk immediately
        chunk_fname = os.path.join(out_dir, f"{model_tag}_emb_chunk_{batch_idx:05d}.npy")
        np.save(chunk_fname, cls_embeddings)
        chunk_files.append(chunk_fname)

        processed += cls_embeddings.shape[0]
        # print a lightweight progress summary every 100 batches
        if (batch_idx + 1) % 100 == 0 or (batch_idx + 1) == total_batches:
            elapsed = time.time() - start_time
            avg_per_batch = elapsed / (batch_idx + 1)
            remaining_batches = total_batches - (batch_idx + 1)
            eta = avg_per_batch * remaining_batches
            print(f"{model_tag} processed {processed}/{len(text_list)} rows — elapsed {elapsed:.1f}s, ETA {eta/60:.2f}min")

    total_time = time.time() - start_time
    print(f"\n{model_tag} finished. Total time: {total_time:.2f} seconds ({total_time/60:.2f} minutes).")
    return chunk_files, total_time

# ---------------- Run embeddings for both models and save chunks ----------------
all_times = {}

# Clinical ModernBERT (CMB)
cmb_chunks, cmb_time = embed_and_save_in_batches(
    full_texts, tokenizer1, CMB, model_tag="CMB", batch_size=BATCH_SIZE, out_dir=OUT_DIR
)
all_times['CMB'] = cmb_time

# BioClinical ModernBERT (BMB)
bmb_chunks, bmb_time = embed_and_save_in_batches(
    full_texts, tokenizer2, BMB, model_tag="BMB", batch_size=BATCH_SIZE, out_dir=OUT_DIR
)
all_times['BMB'] = bmb_time

# ---------------- Concatenate chunk files into final single .npy (streaming) ----------------
def concatenate_chunks(chunk_files, final_path):
    # load first to get dtype & shape
    if not chunk_files:
        raise ValueError("No chunk files to concatenate.")
    # compute total rows
    total_rows = 0
    for f in chunk_files:
        arr = np.load(f, mmap_mode='r')
        total_rows += arr.shape[0]
    # infer dim from first file
    first = np.load(chunk_files[0], mmap_mode='r')
    emb_dim = first.shape[1]

    # create memmap to write final output without storing all in memory
    final_np = np.memmap(final_path, dtype=first.dtype, mode='w+', shape=(total_rows, emb_dim))
    pos = 0
    for f in chunk_files:
        arr = np.load(f)
        final_np[pos: pos + arr.shape[0]] = arr
        pos += arr.shape[0]
    # flush to disk
    del final_np

# final filenames
cmb_final = os.path.join(OUT_DIR, "cmb_embeddings_full.npy")
bmb_final = os.path.join(OUT_DIR, "bmb_embeddings_full.npy")

print("\nConcatenating CMB chunks into final file...")
conc_start = time.time()
concatenate_chunks(cmb_chunks, cmb_final)
conc_time_cmb = time.time() - conc_start
print(f"CMB concatenation time: {conc_time_cmb:.2f} sec. Final file: {cmb_final}")

print("\nConcatenating BMB chunks into final file...")
conc_start = time.time()
concatenate_chunks(bmb_chunks, bmb_final)
conc_time_bmb = time.time() - conc_start
print(f"BMB concatenation time: {conc_time_bmb:.2f} sec. Final file: {bmb_final}")

# ---------------- Summary & verification ----------------
total_time = all_times['CMB'] + all_times['BMB'] + conc_time_cmb + conc_time_bmb
print("\n----- SUMMARY -----")
print(f"Samples embedded: {n_samples:,}")
print(f"CMB embedding time: {all_times['CMB']:.2f} s ({all_times['CMB']/60:.2f} min)")
print(f"BMB embedding time: {all_times['BMB']:.2f} s ({all_times['BMB']/60:.2f} min)")
print(f"CMB concatenation time: {conc_time_cmb:.2f} s")
print(f"BMB concatenation time: {conc_time_bmb:.2f} s")
print(f"Total elapsed time (including concatenation): {total_time/60:.2f} minutes")

# quick shape check
cmb_arr = np.load(cmb_final, mmap_mode='r')
bmb_arr = np.load(bmb_final, mmap_mode='r')
print("\nFinal shapes (memmap):")
print("CMB:", cmb_arr.shape)
print("BMB:", bmb_arr.shape)


Using device: cuda
Total texts to embed (train+validate+test): 1,292,579


CMB batches:   1%|          | 103/20197 [00:05<16:07, 20.77it/s]

CMB processed 6400/1292579 rows — elapsed 4.9s, ETA 16.32min


CMB batches:   1%|          | 202/20197 [00:09<16:15, 20.49it/s]

CMB processed 12800/1292579 rows — elapsed 9.7s, ETA 16.21min


...

BMB batches: 100%|█████████▉| 20103/20197 [19:17<00:05, 17.51it/s]

BMB processed 1286400/1292579 rows — elapsed 1157.8s, ETA 0.09min


BMB batches: 100%|██████████| 20197/20197 [19:23<00:00, 17.36it/s]


BMB processed 1292579/1292579 rows — elapsed 1163.4s, ETA 0.00min

BMB finished. Total time: 1163.45 seconds (19.39 minutes).

Concatenating CMB chunks into final file...
CMB concatenation time: 14.56 sec. Final file: /kaggle/working/embeddings/cmb_embeddings_full.npy

Concatenating BMB chunks into final file...
BMB concatenation time: 15.28 sec. Final file: /kaggle/working/embeddings/bmb_embeddings_full.npy

----- SUMMARY -----
Samples embedded: 1,292,579
CMB embedding time: 950.34 s (15.84 min)
BMB embedding time: 1163.45 s (19.39 min)
CMB concatenation time: 14.56 s
BMB concatenation time: 15.28 s
Total elapsed time (including concatenation): 35.73 minutes


ValueError: Cannot load file containing pickled data when allow_pickle=False

In [1]:
import os
import shutil

# Path where your final embeddings are
embeddings_dir = "/kaggle/working/embeddings"

# Check if files are there
print("Files in embeddings_dir:", os.listdir(embeddings_dir))

# Move them to /kaggle/working (Kaggle only lets you create dataset from /kaggle/working root)
for fname in os.listdir(embeddings_dir):
    shutil.copy(os.path.join(embeddings_dir, fname), f"/kaggle/working/{fname}")

print("Copied to /kaggle/working for dataset saving.")


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/embeddings'